In [2]:
%load_ext autoreload
%autoreload 2

In [72]:
from futures.llm import LLM
from futures.futurist import Futurist
from futures.db import DB
import pandas as pd
from futures.futurist import Futurist
from futures.substack import SubstackManager
from collections import Counter
from langchain_openai import ChatOpenAI
import json

In [ ]:
llm = LLM(datastore="data/")
sm = SubstackManager()
f = Futurist(llm)
db = DB(llm)
vectordb = db.createDB()

llm.ai = ChatOpenAI(model="gpt-4o-mini")

client=<openai.resources.embeddings.Embeddings object at 0x79b49a4b5750> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x79b49a55f5e0> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
# VectorDB: continue on the DB
- VectorDB:  4091 elements already stored.
- VectorDB: adding 3252 documents.


In [57]:
db.getArticles(topic="Trump security", k =2)

[Document(id='f331a0bd-b990-4567-8690-7c08bcb9307a', metadata={'LEN': '9913', 'author': 'kelu', 'keywords': "['Supreme Court' 'ex-presidents' 'immunity' 'criminal prosecution'\n 'pre-election Trump trial' 'Washington criminal case' 'Donald Trump'\n 'presidential power' 'official acts' 'political divide']", 'origin': '20040714', 'source': 'https://apnews.com/article/supreme-court-trump-capitol-riot-immunity-2dc0d1c2368d404adc0054151490f542', 'src': '5cbf19bc5097424a9bd615c891be2a12', 'summary': 'The Supreme Court has ruled that former presidents have broad immunity from criminal prosecution for official acts, effectively delaying the Washington criminal case against ex-President Donald Trump. This ruling highlights a muscular view of presidential power, with the majority of conservative justices narrowing the case against Trump. This decision has sparked political divide, with dissenting judges criticizing it for undermining the principle that no person is above the law. The ruling also

# Read memory

In [14]:
meta = pd.read_parquet(llm.srcMetas).reset_index(drop=True)
signals = pd.read_parquet(llm.srcSeeds).reset_index(drop=True)
summaries = pd.read_parquet(llm.srcSummaries).reset_index(drop=True)
df = pd.read_parquet(llm.srcArticles)
df.columns = ["src","content","origin","LEN"]
techs = pd.read_parquet(llm.srcEmergingTechs).reset_index(drop=True)
behav = pd.read_parquet(llm.srcEmergingBehav).reset_index(drop=True)
issue = pd.read_parquet(llm.srcEmergingIssue).reset_index(drop=True)
cncrn = pd.read_parquet(llm.srcEmergingConcerns).reset_index(drop=True)

In [15]:
meta["path"] = meta["path"].apply(lambda row: row.split("/")[-1].split(".")[0].replace("d",""))
good = meta[meta["path"].str.startswith("2025")].src.tolist()
print(len(good),"articles in 2025")

1119 articles in 2025


In [23]:
themes = ", ".join(meta[meta.src.isin(good)].themes.tolist()).split(", ")
themes = [x.strip().lower() for x in themes]
counts = Counter(themes)

# Categorisation

In [37]:
S = "* "+"\n* ".join(signals[(signals.src.isin(good)) & (signals.relevancy > 4)].description)
categories  = llm.ai.invoke("You are a futurist. Read the list of signals below. You will have to find 20 categories and trends of change that cover these signals as much as possible:\n\n"+S).content
categories = categories.split("\n\n")[1:-1]
trends = {}
for cat in categories:
    title = cat.split("**  \n   ")[0][5:]
    desc = "\n".join(cat.split("**  \n   ")[1:])
    trends[title] = desc
twenty_trends = trends

In [75]:
with open("data/2025.json","w") as f:
    f.write(json.dumps(twenty_trends, indent=4))

In [76]:
categories_tags = []
summaries = summaries[summaries.src.isin(good)].reset_index(drop=True)
for ix, row in summaries.iterrows():
    if ix ==4000 :
        break
    if not ix%100:
        print(ix/len(summaries)*100,"% done")
    txt = row.title + ": "+ row.summary

    #print(row.title)
    CATS = llm.ai.invoke("Read the text below, and find up to three trends to that this text illustrates. You can pick 0 to 3 trends.\
                   The categories are these: \n\n"+ \
                  "* "+"\n* ".join(list(twenty_trends.keys())) +"" + \
                  "\n\n"+txt+"\n\nYOU CAN ONLY PICK ONE TO THREE TRENDS, answer only with one line with trend with its name. Only the the names you have been given, and do not change them nor add to them. If no trend, answer None.").content
    CATS = CATS.replace("*","").split("\n")
    CATS = [x.strip() for x in CATS if len(x.strip())>0 and x.strip().lower() != "none"]
    categories_tags.append( (row.src, CATS) )
    #print(CATS)
categories_tags = pd.DataFrame(categories_tags, columns=["src","categories"])

0.0 % done
9.066183136899365 % done
18.13236627379873 % done
27.198549410698096 % done
36.26473254759746 % done
45.33091568449682 % done
54.39709882139619 % done
63.46328195829556 % done
72.52946509519492 % done
81.59564823209429 % done
90.66183136899365 % done
99.72801450589301 % done


In [ ]:
from futures.fcts import getEmergingConcerns
from futures.fcts import getWeaksignals, getEmergingBehaviors, getEmergingTechnologies


In [ ]:
todo = summaries.merge(categories_tags, on="src")


MD = "# Review of 2025!!\n\n## Raw - Yearly summaries per trend\n\n"
for k in twenty_trends:
    
    topic = twenty_trends[k]
    k = k.replace("*","")
    filtered_df = todo[todo['categories'].apply(lambda x: k in x)]
    n = len(filtered_df)
    
    if n>70:
        interest = filtered_df.src.unique()
        MD += f"\n\n## Trend: {k}\n\n"
        print(len(interest),"articles for trend:",k)

        MD += f"There were {n} articles reviewed.\n\n"
        signals = pd.read_parquet(llm.srcSeeds).reset_index(drop=True)
        behav = pd.read_parquet(llm.srcEmergingBehav).reset_index(drop=True)
        techs = pd.read_parquet(llm.srcEmergingTechs).reset_index(drop=True)
        cncrn = pd.read_parquet(llm.srcEmergingConcerns).reset_index(drop=True)
        # Keeping the interesting parts
        signals = signals[signals.src.isin(interest)].reset_index(drop=True)
        signals = signals[signals.relevancy > 3].reset_index(drop=True)
        behav = behav[behav.src.isin(interest)].reset_index(drop=True)
        techs = techs[techs.src.isin(interest)].reset_index(drop=True)
        cncrn = cncrn[cncrn.src.isin(interest)].reset_index(drop=True)

        MD += "\n\n### Weak Signals\n\n"
        SIGNALS = signals[signals.columns[:-3]].to_markdown()
        md_signals = llm.askTool("Act as a futurist / prospectivist who is working on a year's worth of inputs and tries to write a yearly summary. Identify signals in this table and summarize them in around the main global/overarching 10 signals capturing the range of the table content.", SIGNALS[:900000], getWeaksignals(), "get_weaksignals")
        md_signals = md_signals[md_signals.columns[:-1]].to_markdown()
        MD += "\n\n"+md_signals +"\n\n"
        MD += "\n\n### Concerns\n\n"
        CCRN = cncrn[cncrn.columns[:-3]].to_markdown()
        md_concerns = llm.askTool("Act as a futurist / prospectivist who is working on a year's worth of inputs and tries to write a yearly summary. Identify concerns in this table and summarize them in around the main 10 concerns", CCRN[:900000], getEmergingConcerns(), "get_emergingconcerns")
        md_concerns = md_concerns[md_concerns.columns[:-1]].to_markdown()
        MD += "\n\n"+md_concerns +"\n\n"
        MD += "\n\n### Techs\n\n"
        TECHS = techs[techs.columns[:-3]].to_markdown()
        md_tech = llm.askTool("Act as a futurist / prospectivist who is working on a year's worth of inputs and tries to write a yearly summary. Identify technologies in this table and summarize them in around the main 10 techs", TECHS[:900000], getEmergingTechnologies(), "get_emergingtechnologies")
        md_tech = md_tech[md_tech.columns[:-1]].to_markdown()
        MD += "\n\n"+md_tech +"\n\n"
        MD += "\n\n### Behaviors\n\n"   
        BEHAV = behav[behav.columns[:-3]].to_markdown()
        md_behav = llm.askTool("Act as a futurist / prospectivist who is working on a year's worth of inputs and tries to write a yearly summary. Identify behaviors in this table and summarize them in around the main 10 human behaviors capturing a synthesis of global/overarching range of behaviors, focusing only on this table and trying to give back something that encapsulates the main points..", BEHAV[:900000], getEmergingBehaviors(), "get_emergingbehaviors")
        md_behav = md_behav[md_behav.columns[:-1]].to_markdown()
        MD += "\n\n"+md_behav +"\n\n"


with open("docs/analyses/yearly/2025.md","w") as f:
    f.write(MD)


485 articles for trend: Technological Advancements in AI and Robotics
--- Function call:	 2025-12-21 16:24:39
--- Function call:	 2025-12-21 16:24:39
--- Function call:	 2025-12-21 16:24:39
--- Function call:	 2025-12-21 16:24:39
270 articles for trend: Consumer Behavior and Values Shift
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:40
82 articles for trend: Cultural and Historical Narratives
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:40
136 articles for trend: Political Polarization and Populism
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:40
--- Function call:	 2025-12-21 16:24:41
--- Function call:	 2025-12-21 16:24:41
181 articles for trend: Environmental Awareness and Action
--- Function call:	 2025-12-21 16:24:41
--- Function c

# Writing

--- Function call:	 2025-12-21 15:43:07
|    | name                                             | description                                                                                                                        |
|---:|:-------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------|
|  0 | AI-Enhanced Geothermal Resource Identification   | Utilizing AI to improve the identification of geothermal energy resources.                                                         |
|  1 | Deep-drilling Technologies for Geothermal Energy | Advancements in drilling techniques to increase the efficiency of geothermal energy extraction.                                    |
|  2 | Galactic Security Solutions                      | Innovative approaches to ensure security in space activities and operations.                                                       |
|  3 